In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///snorkel'

from snorkel import SnorkelSession

session = SnorkelSession()
from snorkel.parser import XMLMultiDocPreprocessor

# The following line is for testing only. Feel free to ignore it.

file_path = 'articles/TROP2.xml'
TROP2_preprocessor = XMLMultiDocPreprocessor(
    path=file_path,
    doc='.//Article',
    text='./text/text()',
    id='./article-id/text()'
)

In [2]:
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser()
corpus_parser.apply(list(TROP2_preprocessor), parallelism=4) #parallelism can be run with a Postgres DBMS, but not SQLite

1
4163115A
2
4259449A
3
3778290A
4
4375832A
5
3126726A
6
3039653A
7
4863686A
8
4437474A
9
4999623A
10
4483763A
11
3410424A
12
3249013A
13
3967411A
14
5042010A
15
2680948A
16
2855629A
17
3743266A
18
4986166A
19
2946292A
20
4253434A
21
5086635A
22
2700972A
23
4829466A
24
4327780A
25
3338700A
26
3938934A
27
4631976A
28
4426451A
29
3673221A
30
3092477A
31
4251850A
32
2939361A
33
2939361B
34
3514740A
35
4186637A
36
3739120A
37
4972371A
38
3866276A
39
4546469A
40
4295802A
41
3224189A
42
4505916A
43
5355237A
44
3786374A
45
4183706A
46
3756971A
47
4882422A
48
4467439A
49
5485361A
50
5502951A
51
3215704A
52
3885472A
53
4306156A
54
3934101A
55
5414739A
56
3378878A
57
2607235A
58
4410184A
59
5492016A
60
3981717A
61
4292049A
62
3233648A
63
5438686A
64
3583404A
65
3991172A
66
5378268A
67
4154955A
68
3867924A
69
3410494A
70
2783355A
71
4096804A
72
4868745A
73
4074497A
74
4621003A
75
5505023A
76
2634835A
77
5289527A
78
3718634A
79
4756367A
80
3474961A
81
4685746A
82
2743239A
83
5045392A
84
4011766A
8

In [ ]:
from six.moves.cPickle import load
from snorkel.models import Document, Sentence
import cPickle

with open('articles/doc_ids.pkl', 'rb') as f:
    train_ids, dev_ids, test_ids = load(f)
train_ids, dev_ids, test_ids = set(train_ids), set(dev_ids), set(test_ids)
print len(train_ids)
print len(dev_ids)
print len(test_ids)
train_sents, dev_sents, test_sents = set(), set(), set()
docs = session.query(Document).order_by(Document.name).all()
for i, doc in enumerate(docs):
    for s in doc.sentences:
        if doc.name in train_ids:
            train_sents.add(s)
        elif doc.name in dev_ids:
            dev_sents.add(s)
        elif doc.name in test_ids:
            test_sents.add(s)
        else:
            raise Exception('ID <{0}> not found in any id set'.format(doc.name))
    

In [3]:
from snorkel.models import Candidate, candidate_subclass

BiomarkerCondition = candidate_subclass('BiomarkerCondition', ['biomarker', 'condition'])
BiomarkerType = candidate_subclass('BiomarkerType', ['biomarker', 'typ3'])
BiomarkerMedium = candidate_subclass('BiomarkerMedium', ['biomarker', 'medium'])
BiomarkerLevelUnit = candidate_subclass('BiomarkerLevelUnit', ['biomarker', 'level', 'unit'])
BiomarkerDrug = candidate_subclass('BiomarkerDrug', ['biomarker', 'drug'])


from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher
import matchers
from snorkel.models import Document
from snorkel.viewer import SentenceNgramViewer
import os

biomarker_ngrams = Ngrams(n_max=1)
condition_ngrams = Ngrams(n_max=7)
type_ngrams = Ngrams(n_max=5)
medium_ngrams = Ngrams(n_max=5)
level_ngrams = Ngrams(n_max=8)
unit_ngrams = Ngrams(n_max=8)
drug_ngrams = Ngrams(n_max=5)

# Create our two Matchers
bMatcher = matchers.getBiomarkerMatcher()
cMatcher = matchers.getDiseaseMatcher()
tMatcher = matchers.getTypeMatcher()
mMatcher = matchers.getMediumMatcher()
lMatcher = matchers.getLevelsMatcher()
uMatcher = matchers.getUnitsMatcher()
dMatcher = matchers.getDrugMatcher()


# Building the CandidateExtractor 
bc_candidate_extractor = CandidateExtractor(BiomarkerCondition, [biomarker_ngrams, condition_ngrams], [bMatcher, cMatcher])
bt_candidate_extractor = CandidateExtractor(BiomarkerType, [biomarker_ngrams, type_ngrams], [bMatcher, tMatcher])
bm_candidate_extractor = CandidateExtractor(BiomarkerMedium, [biomarker_ngrams, medium_ngrams], [bMatcher, mMatcher])
blu_candidate_extractor = CandidateExtractor(BiomarkerLevelUnit, [biomarker_ngrams, level_ngrams, unit_ngrams], [bMatcher, lMatcher, uMatcher])
bd_candidate_extractor = CandidateExtractor(BiomarkerDrug, [biomarker_ngrams, drug_ngrams], [bMatcher, dMatcher])

extractors = [bc_candidate_extractor, bt_candidate_extractor, bm_candidate_extractor, blu_candidate_extractor, bd_candidate_extractor]

ASDFASDF


In [9]:
docs = session.query(Document).order_by(Document.name).all()
sents = []
for doc in docs:
    for s in doc.sentences:
        sents.append(s)
print len(sents)
for extractor in extractors:
    extractor.apply(sents, split=0, clear=False)
    session.commit()
#     hasCompleted = False
#     while(not hasCompleted):
#         try:
#             extractor.apply(sents, split=0, clear=False)
#             session.commit()
#             hasCompleted = True
#         except:
#             print "RIP"
    




71673
Running UDF...
[=                                       ] 1%

KeyboardInterrupt: 

In [5]:
from playsound import playsound
playsound('ding.mp3')

In [12]:
import candidate_adjective_fixer
import candidate_adjective_fixer_medium
import candidate_adjective_fixer_drug 

c_dev_cands = session.query(BiomarkerCondition).filter(BiomarkerCondition.split == 0).all()
m_dev_cands = session.query(BiomarkerMedium).filter(BiomarkerMedium.split == 0).all()
d_dev_cands = session.query(BiomarkerDrug).filter(BiomarkerDrug.split == 0).all()

print len(c_dev_cands)
print len(m_dev_cands)
print len(d_dev_cands)

candidate_adjective_fixer_drug.add_adj_candidate(session, BiomarkerDrug, d_dev_cands, 0)
candidate_adjective_fixer_medium.add_adj_candidate(session, BiomarkerMedium, m_dev_cands, 0)
candidate_adjective_fixer.add_adj_candidate(session, BiomarkerCondition, c_dev_cands, 0)


21657
4835
9464
additional  added to iron
Checking Biomarker: 1247656A::span:32381:32382
Checking Condition: 1247656A::span:37:51
1
 added to iron
additional  added to iron
Checking Biomarker: 1247656A::span:32385:32388
Checking Condition: 1247656A::span:37:51
1
 added to iron
 added to Luminal
 added to Herceptin
antibody therapeutic  added to trastuzumab
Checking Biomarker: 1523209A::span:11085:11090
Checking Condition: 1523209A::span:191:222
1
 added to Herceptin
antibody therapeutic  added to trastuzumab
Checking Biomarker: 1523209A::span:11143:11145
Checking Condition: 1523209A::span:191:222
1
 added to Herceptin
antibody therapeutic  added to trastuzumab
Checking Biomarker: 1523209A::span:11207:11210
Checking Condition: 1523209A::span:191:222
1
 added to Herceptin
antibody therapeutic  added to trastuzumab
Checking Biomarker: 1523209A::span:11226:11229
Checking Condition: 1523209A::span:191:222
1
 added to Herceptin
antibody therapeutic  added to trastuzumab
Checking Biomarker: 1

1
concurrent  added to cisplatin
Checking Biomarker: 2634835A::span:2756:2759
Checking Condition: 2634835A::span:84:103
1
concurrent  added to cisplatin
Checking Biomarker: 2634835A::span:2581:2585
Checking Condition: 2634835A::span:84:103
1
concurrent  added to cisplatin
Checking Biomarker: 2634835A::span:2267:2270
Checking Condition: 2634835A::span:84:103
1
concurrent  added to cisplatin
Checking Biomarker: 2634835A::span:2749:2753
Checking Condition: 2634835A::span:84:103
1
concurrent  added to cisplatin
Checking Biomarker: 2634835A::span:2770:2774
Checking Condition: 2634835A::span:84:103
1
 added to Cetuximab
 added to Cetuximab
 added to Cetuximab
 added to cetuximab
 added to Cetuximab
 added to cetuximab
 added to Cetuximab
 added to cetuximab
 added to Cetuximab
 added to cetuximab
 added to Cetuximab
 added to cetuximab
 added to Cetuximab
 added to cetuximab
 added to Cetuximab
 added to cetuximab
 added to Cetuximab
 added to cetuximab
 added to cetuximab
 added to cetuxima

 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to thyroid
unesterified  added to cholesterol
Checking Biomarker: 2695576A::span:50133:50135
Checking Condition: 2695576A::span:76:99
1
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to thyroid
 added to thyroid
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cholesterol
 added to cho

 added to progesterone
 added to progesterone
 added to progesterone
 added to progesterone
 added to progesterone
 added to progesterone
mg/ml  added to insulin
Checking Biomarker: 2951427A::span:24552:24555
Checking Condition: 2951427A::span:157:169
1
mg/ml  added to insulin
Checking Biomarker: 2951427A::span:24541:24543
Checking Condition: 2951427A::span:157:169
1
mg/ml  added to insulin
Checking Biomarker: 2951427A::span:24561:24563
Checking Condition: 2951427A::span:157:169
1
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to Trypsin
 added to Trypsin
 added to Trypsin
 added to retroorbital
 added to retroorbital
 added to cetuximab
 added to trastuzumab
 added to trastuzumab
 added to trastuzumab
 added to sorafenib
 added to cetuximab
 added to gefitinib
 added to gefitinib
 added to gefitinib
 added to gefitinib
 added to gefitinib
 added to gefitinib
 added to imatinib
 added to imatinib
 added to imatinib
 added to imatinib
 added to imatinib
 added t

Checking Biomarker: 3110956A::span:23597:23598
Checking Condition: 3110956A::span:285:303
1
ng/ml  added to oncostatin
Checking Biomarker: 3110956A::span:23682:23683
Checking Condition: 3110956A::span:158:173
1
mol/l  added to dexamethasone
Checking Biomarker: 3110956A::span:23682:23683
Checking Condition: 3110956A::span:285:303
1
 added to Excel
 added to Albumin
 added to Albumin
 added to tamoxifen
 added to tamoxifen
 added to tamoxifen
 added to tamoxifen
 added to tamoxifen
mg/ml  added to Collagenase
Checking Biomarker: 3110957A::span:29658:29665
Checking Condition: 3110957A::span:92:108
1
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to BD
 added to Methylene Blue
 added to Methylene Blue
 added to Excel
 added to Excel
 added to fluorouracil
mm  added to glutamine
Checking

...

Checking Biomarker: 5437009A::span:17895:17897
Checking Condition: 5437009A::span:38:61
1
Checking Biomarker: 5437009A::span:17819:17823
Checking Condition: 5437009A::span:38:61
1
Checking Biomarker: 5437009A::span:18342:18345
Checking Condition: 5437009A::span:50:67
1
Checking Biomarker: 5437009A::span:18263:18267
Checking Condition: 5437009A::span:50:67
1
Checking Biomarker: 5437009A::span:18356:18359
Checking Condition: 5437009A::span:50:67
1
Checking Biomarker: 5437009A::span:18313:18317
Checking Condition: 5437009A::span:50:67
1
Checking Biomarker: 5437009A::span:18443:18447
Checking Condition: 5437009A::span:143:166
1
Checking Biomarker: 5437009A::span:18504:18508
Checking Condition: 5437009A::span:143:166
1
Checking Biomarker: 5437009A::span:18417:18421
Checking Condition: 5437009A::span:143:166
1
Checking Biomarker: 5437009A::span:18589:18593
Checking Condition: 5437009A::span:53:70
1
Checking Biomarker: 5437009A::span:18692:18696
Checking Condition: 5437009A::span:53:70
1
Chec

1
Checking Biomarker: 5438686A::span:10147:10151
Checking Condition: 5438686A::span:285:305
1
Checking Biomarker: 5442765A::span:15481:15482
Checking Condition: 5442765A::span:88:108
1
Checking Biomarker: 5442765A::span:15727:15729
Checking Condition: 5442765A::span:88:108
1
Checking Biomarker: 5442765A::span:15391:15394
Checking Condition: 5442765A::span:88:108
1
Checking Biomarker: 5442765A::span:15457:15458
Checking Condition: 5442765A::span:88:108
1
Checking Biomarker: 5442765A::span:15361:15364
Checking Condition: 5442765A::span:88:108
1
Checking Biomarker: 5442765A::span:15746:15748
Checking Condition: 5442765A::span:88:108
1
Checking Biomarker: 5442765A::span:29591:29594
Checking Condition: 5442765A::span:36:56
1
Checking Biomarker: 5442765A::span:31317:31320
Checking Condition: 5442765A::span:17:38
1
Checking Biomarker: 5442765A::span:34287:34291
Checking Condition: 5442765A::span:65:80
1
Checking Biomarker: 5442765A::span:34477:34479
Checking Condition: 5442765A::span:65:80
1


Checking Biomarker: 5485361A::span:5748:5751
Checking Condition: 5485361A::span:338:362
1
Checking Biomarker: 5485361A::span:12276:12280
Checking Condition: 5485361A::span:171:182
1
Checking Biomarker: 5485361A::span:12405:12408
Checking Condition: 5485361A::span:171:182
1
Checking Biomarker: 5485361A::span:17329:17330
Checking Condition: 5485361A::span:142:171
1
Checking Biomarker: 5485361A::span:17329:17330
Checking Condition: 5485361A::span:62:108
1
Checking Biomarker: 5485361A::span:18080:18084
Checking Condition: 5485361A::span:176:199
1
Checking Biomarker: 5485361A::span:18080:18084
Checking Condition: 5485361A::span:78:96
1
Checking Biomarker: 5485361A::span:18122:18124
Checking Condition: 5485361A::span:176:199
1
Checking Biomarker: 5485361A::span:18122:18124
Checking Condition: 5485361A::span:78:96
1
Checking Biomarker: 5485361A::span:18112:18116
Checking Condition: 5485361A::span:176:199
1
Checking Biomarker: 5485361A::span:18112:18116
Checking Condition: 5485361A::span:78:96

Checking Condition: 5502951A::span:68:83
1
Checking Biomarker: 5502951A::span:12932:12934
Checking Condition: 5502951A::span:68:83
1
Checking Biomarker: 5502951A::span:14836:14839
Checking Condition: 5502951A::span:225:253
1
Checking Biomarker: 5502951A::span:14791:14793
Checking Condition: 5502951A::span:225:253
1
Checking Biomarker: 5502951A::span:14617:14619
Checking Condition: 5502951A::span:225:253
1
Checking Biomarker: 5502951A::span:14636:14637
Checking Condition: 5502951A::span:225:253
1
Checking Biomarker: 5502951A::span:14622:14623
Checking Condition: 5502951A::span:225:253
1
Checking Biomarker: 5502951A::span:14738:14740
Checking Condition: 5502951A::span:225:253
1
Checking Biomarker: 5502951A::span:16727:16731
Checking Condition: 5502951A::span:120:141
1
Checking Biomarker: 5502951A::span:16736:16740
Checking Condition: 5502951A::span:120:141
1
Checking Biomarker: 5502951A::span:18452:18456
Checking Condition: 5502951A::span:791:833
1
Checking Biomarker: 5502951A::span:1911

Checking Biomarker: 5505023A::span:934:936
Checking Condition: 5505023A::span:89:107
1
Checking Biomarker: 5505023A::span:5480:5482
Checking Condition: 5505023A::span:223:236
1
Checking Biomarker: 5505023A::span:5582:5584
Checking Condition: 5505023A::span:223:236
1
Checking Biomarker: 5505023A::span:5442:5444
Checking Condition: 5505023A::span:223:236
1
Checking Biomarker: 5505023A::span:5523:5525
Checking Condition: 5505023A::span:223:236
1
Checking Biomarker: 5505023A::span:5402:5404
Checking Condition: 5505023A::span:223:236
1
Checking Biomarker: 5513028A::span:331:333
Checking Condition: 5513028A::span:56:114
1
Checking Biomarker: 5513028A::span:331:333
Checking Condition: 5513028A::span:123:147
1
Checking Biomarker: 5513028A::span:331:333
Checking Condition: 5513028A::span:156:180
1
Checking Biomarker: 5513028A::span:331:333
Checking Condition: 5513028A::span:224:247
1
Checking Biomarker: 5513028A::span:398:400
Checking Condition: 5513028A::span:56:114
1
Checking Biomarker: 55130

In [15]:

session.commit()

TypeError: 'long' object is not iterable

In [31]:
# sesion.
from snorkel.lf_helpers import  *
c_dev_cands = session.query(BiomarkerCondition).filter(BiomarkerCondition.split == 0).all()
m_dev_cands = session.query(BiomarkerMedium).filter(BiomarkerMedium.split == 0).all()
d_dev_cands = session.query(BiomarkerDrug).filter(BiomarkerDrug.split == 0).all()
t_dev_cands = session.query(BiomarkerType).filter(BiomarkerType.split == 0).all()
lu_dev_cands = session.query(BiomarkerLevelUnit).filter(BiomarkerLevelUnit.split == 0).all()
allcands = [c_dev_cands, m_dev_cands, d_dev_cands, t_dev_cands]
allTuples = []
for canset in allcands:
    for candidate in canset:
        x = 0
        for thing in (get_between_tokens(candidate)):
            x+=1
        allTuples.append((candidate[0].get_stable_id()[0:str(candidate[0].get_stable_id()).index(':')], candidate[0].get_span(), candidate[1].get_span(), candidate[0].sentence, x))
for candidate in lu_dev_cands: 
    allTuples.append((candidate[0].get_stable_id()[0:str(candidate[0].get_stable_id()).index(':')], candidate[0].get_span(), candidate[1].get_span(), candidate[0].sentence, candidate[2].get_span()))
with open('TROP2_output.tsv', 'wb') as f:
    f.write('')
with open('TROP2_output.tsv', 'a') as f:
    for thing in allTuples:
        tempString = ""
        for val in thing:
            tempString += (str(val) + "\t")
        f.write(tempString + "\n")

In [ ]:
import re
#from snorkel.lf_terms import *
from snorkel.lf_helpers import  *
from snorkel.lf_helpers import get_sent_candidate_spans
from snorkel.lf_helpers import get_left_tokens, get_right_tokens
from random import randint
import cPickle
from PyDictionary import PyDictionary



#umls_dict              = load_umls_dictionary()
#chemicals              = load_chemdner_dictionary()
#abbrv2text, text2abbrv = load_specialist_abbreviations()

keyWords = ["associate", "express", "marker", "biomarker", "elevated", "decreased",
            "correlation", "correlates", "found", "diagnose", "variant", "appear",
            "connect", "relate", "exhibit", "indicate", "signify", "show", "demonstrate",
            "reveal", "suggest", "evidence", "elevation", "indication", "diagnosis",
            "variation", "modification", "suggestion", "link", "derivation", "denote",
            "denotation", "demonstration", "magnification", "depression", "boost", "level",
            "advance", "augmentation", "lessening", "enhancement", "expression", "buildup",
            "diminishing", "diminishment", "reduction", "drop", "dwindling", "lowering"]

negationWords = ["not", "nor", "neither"]

toAdd = []
for keyword in keyWords:
    syns = (PyDictionary().synonym(keyword))
    if not syns == None:
        for syn in syns:
            if not syn in keyWords and not syn in toAdd:
                toAdd.append(syn)
for word in toAdd:
    keyWords.append(word)

markerDatabase = []
with open('databases/markerData.pickle', 'rb') as f:
    markerDatabase = cPickle.load(f)

    
knowAbbreviations = []
with open('databases/abbreviations.com.pkl', 'rb') as f:
    knowAbbreviations = cPickle.load(f)
    

# Biomarker Validity 

def LF_markerDatabase(c):
    if(c.biomarker.get_span() in markerDatabase):
        return 1
    else:
        return 0
def LF_abstract_titleWord(c):
    words_in_between = []
    for thing in get_between_tokens(c):
        words_in_between.append(thing)
    if(len(words_in_between) > 1 and words_in_between[0] == ":"):
        return -1

def LF_single_letter(c):
    if(len(c.biomarker.get_span()) < 2):
        return -1
    else:
        return 0

def LF_known_abs(c):
    if(c.biomarker.get_span() in knowAbbreviations):
        return -1
    
def LF_same_thing(c):
    if(c[0].get_span() == c[1].get_span()):
        return -1
    
def LF_roman_numeral(c):
    biomarker = (c.biomarker.get_span())
    unicodedata.normalize('NFKD', biomarker).encode('ascii','ignore')
    if re.match(r'((?<=\s)|(?<=^))(M{1,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{1,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{1,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{1,3}))(?=\s)',
                biomarker):
        return -1 
    
    
# Disease Specific
    
def LF_distance(c):
    '''<TYPE> cancer'''
    x = 0
    for thing in (get_between_tokens(c)):
        x+=1
    if(x > 8):
        return -1
    else:
        return 1
    '''if(len(get_between_tokens(c)) < 8):
        return 1'''

def LF_keyword(c):
    for keyword in keyWords:
#         print keyword
        if(keyword in get_between_tokens(c)):
            if("not" in get_between_tokens(c)):
                return -1
            else:
                return 1
    return 0
    
def LF_auxpass(c):
    if not 'auxpass' in get_between_tokens(c, attrib='dep_labels'):
        return -1
    else:
        return 1
    
with open('databases/common2000.pkl', 'rb') as f:
    common2000 = cPickle.load(f)

def LF_common_2000(c):
    if(c.condition.get_span() in common2000):
        return -1

# Medium Specific
# Type Specific
# Drug Specific
# Level/Units Specific



In [ ]:
BiomarkerSpecificLFs = [LF_markerDatabase, LF_abstract_titleWord, LF_single_letter, LF_known_abs, LF_same_thing, LF_roman_numeral]
DiseaseSpecificLFs = [LF_distance,  LF_keyword, LF_auxpass, LF_common_2000]
MediumSpecificLFs = []
TypeSpecificLFs = []
DrugSpecificLFs = []
LevelUnitSpecificLFs = []

In [ ]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)

In [ ]:
%time L_train = labeler.apply(split=0)
L_train

In [ ]:
L_train.lf_stats(session)

In [ ]:
from snorkel.learning.structure import DependencySelector
ds = DependencySelector()
deps = ds.select(L_train, threshold=0.1)
len(deps)

In [ ]:
deps

In [ ]:
deps = set()

In [ ]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel(lf_propensity=True)
gen_model.train(
    L_train, deps=deps, decay=0.95, step_size=0.1/L_train.shape[0], reg_param=0.0
)

In [ ]:
train_marginals = gen_model.marginals(L_train)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(train_marginals, bins=20)
plt.show()

In [ ]:
gen_model.learned_lf_stats()

In [ ]:
from snorkel.annotations import save_marginals
save_marginals(session, L_train, train_marginals)

In [ ]:
from load_external_annotations_new import load_external_labels
load_external_labels(session, BiomarkerCondition, 'Biomarker', 'Condition', 'articles/disease_gold_labels.tsv', dev_cands, annotator_name='gold')


In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev
print L_gold_dev


In [ ]:
for thing in L_gold_dev:
    print thing

In [ ]:
L_dev = labeler.apply_existing(split=1)
_ = gen_model.score(session, L_dev, L_gold_dev)

In [ ]:
for candidate in dev_cands:
    print candidate[1].get_stable_id()

In [ ]:
L_dev.lf_stats(session, L_gold_dev, gen_model.learned_lf_stats()['Accuracy'])

In [ ]:
from snorkel.viewer import SentenceNgramViewer
labeled = []
for c in session.query(BiomarkerCondition).filter(BiomarkerCondition.split == 1).all():
    if LF_markerDatabase(c) == 1:
        labeled.append(c)
SentenceNgramViewer(labeled, session, n_per_page=3)

In [ ]:
import numpy as np

# Load dev labels and convert to [0, 1] range
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
dev_labels = (np.ravel(L_gold_dev.todense()) + 1) / 2

In [ ]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

%time F_train = featurizer.apply(split=0)
F_train

In [ ]:
%%time
F_dev  = featurizer.apply_existing(split=1)
F_test = featurizer.apply_existing(split=2)

In [ ]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [ ]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

In [ ]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, F_train, train_marginals, [rate_param, l1_param, l2_param], n=20)

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

import numpy as np
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=0.5, print_freq=25)

### Scoring on the test set



In [ ]:
test_cands = session.query(BiomarkerCondition).filter(BiomarkerCondition.split == 2).all()
train_cands = session.query(BiomarkerCondition).filter(BiomarkerCondition.split == 0).all()

In [ ]:
from load_external_annotations_new import load_external_labels
load_external_labels(session, BiomarkerCondition, 'Biomarker', 'Condition', 'articles/disease_test_labels.tsv', test_cands, annotator_name='gold')

from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

tp, fp, tn, fn = disc_model.score(session, F_test, L_gold_test)

In [ ]:
tp, fp, tn, fn = disc_model.score(session, F_test, L_gold_test)

In [ ]:
print tp

In [ ]:
from snorkel.contrib.rnn import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   50,
    'dropout':    0.5,
    'rebalance':  0.25,
    'print_freq': 5
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, dev_candidates=dev_cands, dev_labels=dev_labels, **train_kwargs)

In [ ]:
lstm.save("biomarkercondition.lstm")